# Imports

In [ ]:
import pandas as pd
import numpy as np

# Variabelen

In [ ]:
bestand = '.\data\dag2.csv'

# Data

In [ ]:
data = pd.read_csv(bestand, header=None)

In [ ]:
data = (data
        .astype('Int64')
)

## Dag 2.1

In [ ]:
def check_data(data: pd.DataFrame):
    """Check given dataframe. All numbers increasing of decreasing en no more than 3 abs difference"""

    data_abs_diff = abs(
            data
            .diff(axis=1)
            .fillna(0)
            )

    data_trans = data.transpose()

    df_check = pd.DataFrame()
    #check stijgen en dalen
    df_check = (
        df_check
        .assign(increase=data_trans.apply(lambda s: s.dropna().is_monotonic_increasing), 
                decrease=data_trans.apply(lambda s: s.dropna().is_monotonic_decreasing),
                increase_or_decrease=lambda df: df.any(axis=1))
        .drop(columns=["increase", "decrease"])     
        
        #Maar is_monotonic_.. neemt ook gelijkblijvende cijfers mee als stijging of daling, die eruit halen:
        .assign(no_constant=~(data.diff(axis=1) == 0).any(axis=1),
                increase_or_decrease=lambda df: df.all(axis=1))
        .drop(columns=["no_constant"])

        #Niets meer dan 3 verschil:
        .assign(no_diff_gt_3=~data_abs_diff.ge(4).any(axis=1),
        
        #En overall beoordeling:        
                all_satisfy=lambda df: df.all(axis=1))
    )
    
    return df_check

In [ ]:
df_check_21 = check_data(data)
#uitkomst
print("Totaal aantal dat voldoet: ")
print(df_check_21["all_satisfy"].sum())

## Dag 2.2

In [ ]:
df_check = df_check_21.copy()

In [ ]:
for col in data.columns:
    data_without_col = data.drop(columns=col)
    df_check[col] = check_data(data_without_col)["all_satisfy"]

In [ ]:
df_check["at_least_one_option"] = (df_check
                                   .drop(columns=["increase_or_decrease","no_diff_gt_3"])
                                   .any(axis=1)
)

In [ ]:
print("Totaal aantal dat voldoet als ergens 1 getal wordt weggelaten: ")
print(df_check["at_least_one_option"].sum())